# Data Science Essentials: Data Cleaning
    <Name>
    <Class>
    <Date>
    

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np


### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. 
Using this dataset, answer the following questions.


1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.

Each part is one point.

In [110]:
df = pd.read_csv("g_t_results.csv")

# Get the percentage of scores that aren't Nan
col_sums = [(col, sum(df[col].value_counts())/len(df[col])) for col in df.columns]
ans_1 = min(col_sums, key = lambda x: x[1])

# Columns that should be numeric that aren't
# print(df.dtypes)
ans_2 = ("OLSAT Verbal Score", "OLSAT Verbal Percentile", "NNAT Non Verbal Raw Score")

# How many third graders have scores outside valid range for OLSAT Verbal Score?
third_graders = df[df['Entering Grade Level'] == '3']
# print(third_graders['OLSAT Verbal Score'].value_counts().sort_index())
ans_3 = 1

# How many data values are missing?
ans_4 = sum(df.isna().sum())

for i, ans in enumerate([ans_1, ans_2, ans_3, ans_4]):
    print("Question {}: {}".format(i+1, ans))

Question 1: ('School Assigned', 0.24786324786324787)
Question 2: ('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')
Question 3: 1
Question 4: 192


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [35]:
df = pd.read_csv("imdb.csv")
print("0. Before:", df.shape)

# Step 1: remove duplicate rows
df = df.drop_duplicates(["movie_title"],keep="last")
print("1. Drop Duplicates:", df.shape)

# Step 2: remove missing data
df = df.dropna()
print("2. Drop Nan:", df.shape)

#Step 3: Remove data outside valid ranges
# Numeric columns:
# duration          30 - 300, a movie has to be at least a half hour
#                             and under 5 hours long. 
# gross             1e3 - 1e9, movies make at least $1000 and less than $1 billion
# title_year        1880 - 2030, Movies didn't exist before 1880's, 
#                                and none are planned for after 2030
# budget            1e3 - 1e9, movies cost at least $1000 and less than $1 billion
# imdb_score        0 - 10, that's the rating scale
# movie_facebook_likes   0 - 7 trillion, at least 0 people liked it, less than 7 
#                                        trillion people liked it
df = df[df['duration'] > 30]
df = df[df['duration'] < 300]
df = df[df['gross'] > 1e3]
df = df[df['gross'] < 1e9]
df = df[df['title_year'] > 1880]
df = df[df['title_year'] < 2030]
df = df[df['budget'] > 1e3]
df = df[df['budget'] < 1e9]
df = df[df['imdb_score'] > 0]
df = df[df['imdb_score'] < 10]
df = df[df['movie_facebook_likes'] > 0]
df = df[df['movie_facebook_likes'] < 7e12]
print("3. Drop invalid data:", df.shape)

# Step 4: Drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.
#print(df.nunique())
drop_cols = df.columns[df.nunique() <= 3]
df = df.drop(drop_cols, axis=1)
print("4. Drop columns with <= 3 different values:", tuple(drop_cols))

# Step 5. Convert the titles to all lower case 
df['movie_title'] = df['movie_title'].str.lower()

df.head()

0. Before: (99, 13)
1. Drop Duplicates: (91, 13)
2. Drop Nan: (63, 13)
3. Drop invalid data: (55, 13)
4. Drop columns with <= 3 different values: ('color', 'language')


,director_name,duration,gross,genres,movie_title,title_year,country,budget,imdb_score,actors,movie_facebook_likes
0,Martin Scorsese,240,116866727.0,Biography|Comedy|Crime|Drama,the wolf of wall street,2013,USA,100000000.0,8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000
1,Shane Black,195,408992272.0,Action|Adventure|Sci-Fi,iron man 3,2013,USA,200000000.0,7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000
2,Quentin Tarantino,187,54116191.0,Crime|Drama|Mystery|Thriller|Western,the hateful eight,2015,USA,44000000.0,7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000
4,Peter Jackson,186,258355354.0,Adventure|Fantasy,the hobbit: the desolation of smaug,2013,USA,225000000.0,7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000
9,Joss Whedon,173,623279547.0,Action|Adventure|Sci-Fi,the avengers,2012,USA,220000000.0,8.1,"Chris Hemsworth,Robert Downey Jr.,Scarlett Joh...",123000


### Problem 3

basketball.csv contains data for all NBA players between 2001 and 2018.
Each row represents a player's stats for a year.

Create two new features:

    career_length (int): number of years player has been playing (start at 0).
    
    target (str): The target team if the player is leaving. If the player is retiring, the target should be 'retires'.
                  A player is retiring if their name doesn't exist the next year.
                  (Set the players in 2019 to NaN).

Remove all duplicate players in each year.
Remove all rows except those where a player changes team, that is, target is not null nor 'retires'.

Drop the player, year, and team_id columns.

Return the first 10 lines of your dataframe and its shape.

In [187]:
df = pd.read_csv('basketball.csv')
print(df.shape)
original_columns = df.columns
grouped = df.groupby('player')
df = df.set_index('player')

# Calculate career length
df['career_length'] = grouped['year'].max() - grouped['year'].min()
df = df.reset_index()

# Figure out target, retired, etc.
df = df.sort_values(['player', 'year'])
df['next_team'] = df.team_id.shift(-1)
df['next_player'] = df.player.shift(-1)
df['same_player'] = df['player'] == df['next_player']
df['same_team'] = df['team_id'] == df['next_team']

same_team_df = df[df['same_player'] == True].query('same_team == True')
same_team_df['target'] = np.nan

change_team_df = df[df['same_player'] == True].query('same_team == False')
change_team_df['target'] = change_team_df['next_team']

retired_df = df[df['same_player'] == False].query('year != 2019')
retired_df['target'] = 'retired'

current_df = df[df['year'] == 2019].query('same_player == False')
current_df['target'] = np.nan

new_df = pd.concat([same_team_df, change_team_df, current_df, retired_df])
new_df = new_df.drop(['same_player', 'same_team', 'next_player', 'next_team'], axis=1)
new_df = new_df.sort_values(['player', 'year'])

# Remove all duplicate players in each year. 
# Remove all rows except those where a player changes team, that is, target is not null nor 'retires'.
new_df = new_df.drop_duplicates(['player', 'year'], keep='first')
new_df = new_df[new_df['target'] != np.nan]
new_df = new_df.dropna()
new_df = new_df[new_df.target != 'retired']

# Drop the player, year, and team_id columns.
new_df = new_df.drop(['player', 'year', 'team_id'], axis=1)
new_df = new_df.sort_index()

# Return the first 10 lines of your dataframe and its shape.
print(new_df.shape)
new_df.head(10)

(4866, 7)
(1197, 6)


,age,per,ws,bpm,career_length,target
14,33,10.5,0.6,-1.1,14,WAS
18,25,4.4,0.0,-6.6,2,WAS
46,28,5.5,-0.3,-4.6,8,MEM
49,32,10.8,0.1,-3.6,11,SAC
62,27,11.0,2.4,-0.6,5,LAL
64,26,15.6,0.6,-0.6,5,DAL
66,27,12.4,0.5,-0.6,7,CLE
70,29,22.3,1.3,6.9,7,PHI
74,27,7.3,-0.1,-4.6,5,MIN
84,36,11.5,0.2,-4.6,17,LAL


### Problem 4

Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    FIXME
	2) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    3) Add a constant column to the dataframe.

    4) Save a copy of the dataframe.

	5) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	6) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model and the summary. Don't print the OLS

In [261]:
df = pd.read_csv('housing.csv')

# Clean up nans
df['LotFrontage'] = df['LotFrontage'].fillna(0.0)
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df['MasVnrArea'] = df['MasVnrArea'].fillna(0.0)
df['Alley'] = df['Alley'].fillna('NA')
df['BsmtQual'] = df['BsmtQual'].fillna('NA')
df['BsmtCond'] = df['BsmtCond'].fillna('NA')
df['BsmtExposure'] = df['BsmtExposure'].fillna('NA')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('NA')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('NA')
df['FireplaceQu'] = df['FireplaceQu'].fillna('NA')
df['GarageType'] = df['GarageType'].fillna('NA')
df['GarageYrBlt'] = df['GarageYrBlt'].fillna('NA')
df['GarageFinish'] = df['GarageFinish'].fillna('NA')
df['GarageQual'] = df['GarageQual'].fillna('NA')
df['GarageCond'] = df['GarageCond'].fillna('NA')
df['PoolQC'] = df['PoolQC'].fillna('NA')
df['Fence'] = df['Fence'].fillna('NA')
df['MiscFeature'] = df['MiscFeature'].fillna('NA')
# Drop Electrical Nan
df = df[df['Electrical'].notna()]

# 2) Identify the variable with nonnumeric values that are misencoded as numbers.
#    One-hot encode it.
# Is it MSSubClass?
df = pd.get_dummies(df, columns=['MSSubClass'], drop_first=True)

# 3) Add a constant column to the dataframe.
df['constant'] = 1

# 4) Save a copy of the dataframe.
df_copy = df

# 5) Choose four categorical featrues that seem very important in predicting SalePrice. 
#    One-hot encode these features and remove all other categorical features.
# MSZoning, ExterCond, Neighborhood, BldgType
df = pd.get_dummies(df, columns=['MSZoning', 'ExterCond', 'Neighborhood', 'BldgType'], 
                    drop_first=True)
df = df._get_numeric_data()

# 6) Run an OLS using all numerical data regression on your model.  
results = sm.OLS(df['SalePrice'], df[df.columns[df.columns != 'SalePrice']]).fit()

results_summary =  results.summary()
results_as_html = results_summary.tables[1].as_html()
df_res = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_res = df_res.sort_values('coef')
print(df_res.tail(10))

                          coef     std err      t  P>|t|       [0.025  \
Neighborhood_BrDale    13930.0    14600.00  0.956  0.339   -14700.000   
Neighborhood_Veenker   14330.0    13000.00  1.105  0.269   -11100.000   
MSZoning_RM            14890.0    11500.00  1.297  0.195    -7633.724   
MSZoning_RL            17260.0    12300.00  1.402  0.161    -6880.701   
MSZoning_RH            18470.0    14800.00  1.244  0.214   -10700.000   
MSZoning_FV            21640.0    14700.00  1.469  0.142    -7255.289   
Neighborhood_NoRidge   40290.0    10600.00  3.795  0.000    19500.000   
Neighborhood_NridgHt   51490.0     9303.08  5.535  0.000    33200.000   
Neighborhood_StoneBr   59060.0    10400.00  5.658  0.000    38600.000   
constant              474900.0  1300000.00  0.365  0.715 -2070000.000   

                         0.975]  
Neighborhood_BrDale     42500.0  
Neighborhood_Veenker    39800.0  
MSZoning_RM             37400.0  
MSZoning_RL             41400.0  
MSZoning_RH             47

### Problem 5

Using the copy of the dataframe you created in Problem 4, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [267]:
df = pd.get_dummies(df_copy, drop_first=True)
print("Shape:", df.shape)
results = sm.OLS(df['SalePrice'], df[df.columns[df.columns != 'SalePrice']]).fit()

results_summary =  results.summary()
results_as_html = results_summary.tables[1].as_html()
df_res = pd.read_html(results_as_html, header=0, index_col=0)[0]
df_res = df_res.sort_values('coef')
print(df_res.tail(10))

Shape: (1459, 371)
                      coef  std err       t  P>|t|    [0.025    0.975]
GarageCond_TA     132400.0  36000.0   3.676  0.000   61700.0  203000.0
GarageCond_Gd     133000.0  37900.0   3.507  0.000   58600.0  207000.0
GarageCond_Po     137400.0  40800.0   3.364  0.001   57300.0  217000.0
RoofMatl_Roll     605800.0  60500.0  10.006  0.000  487000.0  725000.0
RoofMatl_Tar&Grv  618800.0  58100.0  10.644  0.000  505000.0  733000.0
RoofMatl_WdShake  624400.0  56900.0  10.973  0.000  513000.0  736000.0
RoofMatl_CompShg  624700.0  54300.0  11.496  0.000  518000.0  731000.0
RoofMatl_Metal    682600.0  64300.0  10.616  0.000  556000.0  809000.0
RoofMatl_WdShngl  683800.0  55400.0  12.350  0.000  575000.0  792000.0
RoofMatl_Membran  703200.0  64700.0  10.867  0.000  576000.0  830000.0


I think that the previous model is better with less features... This model seems to put a huge amount of emphasis on Garages and Roof materials, which are indicative of other features, such as what neighborhood it's in, etc. A huge house with lots of rooms is more likely to have a fancier roof/garage, so by just adding in those certain things